# Joining scraped courses from Coursera, CodeAcademy and Datacamp and EDA

import libraries

In [4]:
import pandas as pd
from langdetect import detect, DetectorFactory
import numpy as np
from openai import OpenAI
import os as os

read all the datasets

In [24]:
df_codeAcademy = pd.read_csv("codecademy_details.csv")
df_coursera = pd.read_csv("coursera_full.csv")
df_datacamp = pd.read_csv("datacamp_course_details.csv")
df_skills = pd.read_csv("/Users/qianhe/Desktop/bt4103-team20/skillsgraph/processed_devtype_skills_salaries_exp.csv")



In [6]:
print(df_codeAcademy.shape)
df_codeAcademy.head()

(626, 5)


,title,level,duration,description_full,url
0,Prompt Engineering: Generative AI for Malware ...,Beginner,1 hours,"In the dynamic digital landscape, cybersecurit...",https://www.codecademy.com/learn/ext-courses/p...
1,DevSecOps in Azure,Intermediate,1 hours,Microsoft Azure is a collection of various clo...,https://www.codecademy.com/learn/ext-courses/d...
2,Core DevOps Skills: Continuous Principles,Beginner,1 hours,From the moment a developer has finished their...,https://www.codecademy.com/learn/ext-courses/c...
3,Delivering Feedback That Encourages Growth,Beginner,0 hours,The process of giving and receiving feedback i...,https://www.codecademy.com/learn/ext-courses/d...
4,Velo by Wix: Working with Data,Beginner,6 hours,Continue your Wix learning journey with Velo b...,https://www.codecademy.com/learn/velo-by-wix-w...


In [7]:
print(df_coursera.shape)
df_coursera.head()

(21549, 12)


,keyword,title,partner,level,rating,rating_count,duration,what_you_will_learn,skills,recommended_experience,description,url
0,machine learning,Unreal Engine: Master Interactive FX & Game De...,NaN,Advanced,NaN,NaN,1 week,NaN,NaN,NaN,NaN,https://www.coursera.org/learn/unreal-engine-m...
1,machine learning,Compensation: Job Evaluations & Pay Structures,NaN,Beginner,NaN,NaN,9 hour,NaN,NaN,Recommended experience Beginner level No prior...,NaN,https://www.coursera.org/learn/adp-compensatio...
2,machine learning,Linux Foundation Certified System Administrato...,NaN,Intermediate,NaN,NaN,2 hour,NaN,NaN,Recommended experience Intermediate level AWS ...,NaN,https://www.coursera.org/learn/pearson-linux-f...
3,machine learning,Creative Problem Solving,University of Minnesota,Mixed,4.406744,2402.0,1 week,NaN,NaN,NaN,This course deals directly with your ability f...,https://www.coursera.org/learn/creative-proble...
4,machine learning,Financial Modeling & Valuation: Analyze & Fore...,NaN,Advanced,NaN,NaN,2 week,NaN,NaN,NaN,NaN,https://www.coursera.org/learn/financial-model...


In [8]:
print(df_datacamp.shape)
df_datacamp.head()

(550, 5)


,title,level,duration,description_full,url
0,Introduction to Python,Basic,4 hr,Python is a general-purpose programming langua...,https://app.datacamp.com/learn/courses/intro-t...
1,Introduction to SQL,Basic,2 hr,Much of the world's raw data—from electronic m...,https://app.datacamp.com/learn/courses/introdu...
2,Understanding Artificial Intelligence,Basic,2 hr,Artificial Intelligence (AI) is without any do...,https://app.datacamp.com/learn/courses/underst...
3,Introduction to Power BI,Basic,4 hr,Gain a 360° overview of exploring and using Po...,https://app.datacamp.com/learn/courses/introdu...
4,Understanding Prompt Engineering,Basic,1 hr,Embark on an immersive journey to master ChatG...,https://app.datacamp.com/learn/courses/underst...


stack all df

In [9]:
df_coursera = df_coursera.rename(columns={"description": "description_full"})

df_codeAcademy["source"] = "Codecademy"
df_coursera["source"]    = "Coursera"
df_datacamp["source"]    = "DataCamp"

KEEP = ["source", "title", "level", "duration", "description_full", "url"]

combined = pd.concat(
    [df_codeAcademy[KEEP], df_coursera[KEEP], df_datacamp[KEEP]],
    ignore_index=True
)

print(combined.shape)
combined.head()




(22725, 6)


,source,title,level,duration,description_full,url
0,Codecademy,Prompt Engineering: Generative AI for Malware ...,Beginner,1 hours,"In the dynamic digital landscape, cybersecurit...",https://www.codecademy.com/learn/ext-courses/p...
1,Codecademy,DevSecOps in Azure,Intermediate,1 hours,Microsoft Azure is a collection of various clo...,https://www.codecademy.com/learn/ext-courses/d...
2,Codecademy,Core DevOps Skills: Continuous Principles,Beginner,1 hours,From the moment a developer has finished their...,https://www.codecademy.com/learn/ext-courses/c...
3,Codecademy,Delivering Feedback That Encourages Growth,Beginner,0 hours,The process of giving and receiving feedback i...,https://www.codecademy.com/learn/ext-courses/d...
4,Codecademy,Velo by Wix: Working with Data,Beginner,6 hours,Continue your Wix learning journey with Velo b...,https://www.codecademy.com/learn/velo-by-wix-w...


In [10]:
clean = combined.dropna()
print(clean.shape)

(7846, 6)


In [11]:
clean.sample(5)

,source,title,level,duration,description_full,url
11373,Coursera,أساسيات الذكاء الاصطناعي والبيانات الضخمة | AI,Beginner,5 hour,نعيش اليوم في ظل ثورة صناعية جديدة وهي الثورة ...,https://www.coursera.org/learn/artificial-inte...
10016,Coursera,Finanças Orientada a Dados,Beginner,3 week,Nossas boas-vindas ao Curso Finanças Orientada...,https://www.coursera.org/learn/financas-orient...
9813,Coursera,Call Center Customer Service,Beginner,2 week,The Call Center Customer Service course requir...,https://www.coursera.org/learn/call-center-cus...
38,Codecademy,Prompt Engineering Techniques with DeepSeek-R1,Beginner,0 hours,"DeepSeek-R1 can break down problems, consider ...",https://www.codecademy.com/learn/ext-courses/p...
10848,Coursera,Chinese Characters for beginner (3) 汉字(3),Beginner,3 week,This course is aimed at learners who have comp...,https://www.coursera.org/learn/hanzi-3


In [12]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # deterministic

def is_english(text):
    try:
        return detect(text) == "en"
    except Exception:
        return False  # empty/too short/noisy text

mask = clean["description_full"].astype("string").apply(is_english)
only_english = clean[mask].copy()

print(only_english.shape)


(7215, 6)


In [13]:
only_english["level"].unique()


array(['Beginner', 'Intermediate', 'advanced', 'Advanced', 'Mixed',
       'Basic'], dtype=object)

In [14]:
only_english["level"] = only_english["level"].str.strip().replace({"advanced": "Advanced"})
only_english["level"] = only_english["level"].str.strip().replace({"Basic": "Beginner"})
only_english["level"].unique()


array(['Beginner', 'Intermediate', 'Advanced', 'Mixed'], dtype=object)

In [15]:
only_english.sample(5)

,source,title,level,duration,description_full,url
7703,Coursera,Business Models For Sustainability,Beginner,2 week,We can no longer ignore humans’ negative impac...,https://www.coursera.org/learn/business-models...
18330,Coursera,Google Sheets - Advanced Topics,Intermediate,2 hour,This course builds on some of the concepts cov...,https://www.coursera.org/learn/google-sheets--...
7252,Coursera,Document Design,Mixed,3 week,Elevate your document creation and management ...,https://www.coursera.org/learn/document-design
1047,Coursera,What is Data Science?,Beginner,1 week,Do you want to know why data science has been ...,https://www.coursera.org/learn/what-is-datasci...
487,Codecademy,Introduction to Linux,Intermediate,4 hours,"In this introductory course on Linux, we’ll be...",https://www.codecademy.com/learn/introduction-...


OpenAI embeddings

In [16]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# load the SentenceTransformer model (downloads the first time)
MODEL = "all-MiniLM-L6-v2"
st_model = SentenceTransformer(MODEL)

# 1) Make a text column to embed
only_english["text"] = (
    only_english["title"].fillna("") + " — " + only_english["description_full"].fillna("")
).str.strip()

# 2) Embed in batches (SentenceTransformers handles batching internally)
def embed_texts(texts, model=st_model, batch_size=256):
    # normalize_embeddings=True ensures cosine similarity is valid
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return embeddings.astype("float32")

E = embed_texts(only_english["text"].tolist())

# 3) Build FAISS index (cosine similarity via inner product)
dim = E.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(E)

# 4) Append embeddings into DataFrame
only_english["embedding"] = [vec.tolist() for vec in E]


In [18]:
only_english.sample(5)

,source,title,level,duration,description_full,url,text,embedding
5636,Coursera,Effective Programming in Scala,Intermediate,7 week,"Scala is an expressive, versatile, and safe pr...",https://www.coursera.org/learn/effective-scala,Effective Programming in Scala — Scala is an e...,"[-0.006296792067587376, 0.008270658552646637, ..."
5551,Coursera,ESG and Social Activism,Beginner,7 hour,"In this course, you will get an overview of th...",https://www.coursera.org/learn/esg-social-acti...,"ESG and Social Activism — In this course, you ...","[0.026483893394470215, -0.05705971643328667, -..."
999,Coursera,Supervised Machine Learning: Regression and Cl...,Beginner,3 week,In the first course of the Machine Learning Sp...,https://www.coursera.org/learn/machine-learning,Supervised Machine Learning: Regression and Cl...,"[-0.01276035513728857, -0.08604010194540024, 0..."
7904,Coursera,Foundations of User Experience (UX) Design,Beginner,1 week,Foundations of User Experience (UX) Design is ...,https://www.coursera.org/learn/foundations-use...,Foundations of User Experience (UX) Design — F...,"[-0.02999592013657093, -0.005160169210284948, ..."
9344,Coursera,Comfort in Buildings,Intermediate,9 hour,The course provides a comprehensive overview o...,https://www.coursera.org/learn/comfort-in-buil...,Comfort in Buildings — The course provides a c...,"[0.044406458735466, 0.10718964040279388, 0.051..."


In [17]:
only_english.to_csv("only_english.csv", index=False)

In [28]:
only_english.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7215 entries, 0 to 22723
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   source            7215 non-null   object
 1   title             7215 non-null   object
 2   level             7215 non-null   object
 3   duration          7215 non-null   object
 4   description_full  7215 non-null   object
 5   url               7215 non-null   object
 6   text              7215 non-null   object
 7   embedding         7215 non-null   object
dtypes: object(8)
memory usage: 507.3+ KB


In [26]:
df_skills.sample(5)

,DevType,Top_LanguageHaveWorkedWith,Top_DatabaseHaveWorkedWith,Top_PlatformHaveWorkedWith,Top_WebframeHaveWorkedWith,MedianWorkExp,MedianComp
2,"Developer, front-end","JavaScript, HTML/CSS, TypeScript, Bash/Shell (...","PostgreSQL, MySQL, MongoDB, SQLite, MariaDB","npm, Docker, Vite, Webpack, Yarn","React, Node.js, Next.js, Express, Angular",8.0,63808.0
19,Data scientist,"Python, SQL, Bash/Shell (all shells), HTML/CSS, R","PostgreSQL, SQLite, MySQL, Microsoft SQL Serve...","Docker, Pip, Amazon Web Services (AWS), Micros...","FastAPI, Flask, Node.js, React, Django",8.0,87011.0
4,"Developer, full-stack","JavaScript, HTML/CSS, SQL, TypeScript, Python","PostgreSQL, MySQL, SQLite, Microsoft SQL Serve...","Docker, npm, Amazon Web Services (AWS), Vite, Pip","Node.js, React, jQuery, ASP.NET Core, Next.js",10.0,72000.0
16,DevOps engineer or professional,"Bash/Shell (all shells), Python, SQL, JavaScri...","PostgreSQL, MySQL, Redis, SQLite, Elasticsearch","Docker, Kubernetes, Pip, npm, Terraform","Node.js, React, FastAPI, Flask, Angular",12.0,91198.0
15,Cloud infrastructure engineer,"Bash/Shell (all shells), Python, Go, JavaScrip...","PostgreSQL, Redis, MySQL, SQLite, Elasticsearch","Docker, Kubernetes, Terraform, Amazon Web Serv...","Node.js, React, Flask, FastAPI, Express",10.0,109866.0


In [27]:
df_skills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   DevType                     29 non-null     object 
 1   Top_LanguageHaveWorkedWith  29 non-null     object 
 2   Top_DatabaseHaveWorkedWith  29 non-null     object 
 3   Top_PlatformHaveWorkedWith  29 non-null     object 
 4   Top_WebframeHaveWorkedWith  29 non-null     object 
 5   MedianWorkExp               29 non-null     float64
 6   MedianComp                  29 non-null     float64
dtypes: float64(2), object(5)
memory usage: 1.7+ KB


In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import minmax_scale
from sentence_transformers import SentenceTransformer
import faiss

# -----------------------------
# 0) PREP: sanitize DataFrame
# -----------------------------
courses = only_english.reset_index(drop=True).copy()

def parse_duration_to_hours(s):
    if pd.isna(s): return np.nan
    t = str(s).lower(); h = 0.0
    m = re.search(r'(\d+(?:\.\d+)?)\s*h', t); 
    if m: h += float(m.group(1))
    m = re.search(r'(\d+(?:\.\d+)?)\s*m', t); 
    if m: h += float(m.group(1)) / 60
    if h == 0:
        m = re.search(r'(\d+(?:\.\d+)?)\s*week', t)
        if m: h += float(m.group(1)) * 3
    if h == 0:
        m = re.search(r'(\d+(?:\.\d+)?)\s*day', t)
        if m: h += float(m.group(1)) * 6
    return h if h > 0 else np.nan

courses["duration_hours"] = courses["duration"].map(parse_duration_to_hours)
courses["text_lex"] = ((courses["title"].fillna("") + " ") * 2 + courses["description_full"].fillna("")).str.lower()

# -----------------------------
# 1) TF-IDF (lexical index)
# -----------------------------
tfidf = TfidfVectorizer(
    min_df=2, ngram_range=(1, 2),
    token_pattern=r"(?u)\b[A-Za-z\+\#\.\-]+\b"
)
X_lex = tfidf.fit_transform(courses["text_lex"])

# -----------------------------
# 2) Embeddings + FAISS
# -----------------------------
if "embedding" in courses.columns and courses["embedding"].notna().all():
    try:
        E = np.vstack(courses["embedding"].apply(np.array)).astype("float32")
    except Exception:
        import ast
        E = np.vstack(courses["embedding"].apply(lambda x: np.array(ast.literal_eval(x)))).astype("float32")
else:
    st_model_tmp = SentenceTransformer("all-MiniLM-L6-v2")
    E = st_model_tmp.encode(
        (courses["title"].fillna("") + " — " + courses["description_full"].fillna("")).tolist(),
        batch_size=256, convert_to_numpy=True, normalize_embeddings=True
    ).astype("float32")

index = faiss.IndexFlatIP(E.shape[1]); index.add(E)
st_model = SentenceTransformer("all-MiniLM-L6-v2")

# -----------------------------
# 3) Helpers
# -----------------------------
AMBIGUOUS = {
    "r": "R programming language", "go": "Go programming language",
    "c": "C programming language", "sql": "SQL database and queries",
    "aws": "Amazon Web Services", "gcp": "Google Cloud Platform",
    "azure": "Microsoft Azure"
}

def split_skills(s):
    if pd.isna(s) or not str(s).strip(): return []
    return [x.strip() for x in re.split(r"[,/]", str(s)) if x.strip()]

def expand_token(tok): return AMBIGUOUS.get(tok.strip().lower(), tok.strip())

def build_queries_from_row(row):
    devtype = str(row["DevType"]).strip()
    cols = ["Top_LanguageHaveWorkedWith","Top_DatabaseHaveWorkedWith","Top_PlatformHaveWorkedWith","Top_WebframeHaveWorkedWith"]
    raw = []; [raw.extend(split_skills(row[c])) for c in cols]
    seen, skills = set(), []
    for s in raw:
        if s not in seen: seen.add(s); skills.append(s)
    lex_tokens = [expand_token(s) for s in skills]
    q_lex_text = " ".join(lex_tokens + [devtype])
    q_sem_text = f"Best courses for a {devtype} covering {', '.join([expand_token(s) for s in skills[:12]])}."
    return devtype, skills, q_lex_text, q_sem_text

# -----------------------------
# 4) Scoring
# -----------------------------
def score_and_rank_for_row(row, topk=5, candidate_k=1000,
                           w_sem=0.6, w_lex=0.4, w_cov=0.15,
                           title_bonus=0.05):
    devtype, skills, q_lex_text, q_sem_text = build_queries_from_row(row)
    q_emb = st_model.encode([q_sem_text], normalize_embeddings=True).astype("float32")
    D, I = index.search(q_emb, k=min(candidate_k, len(courses)))
    sem_scores = np.zeros(len(courses), dtype="float32"); sem_scores[I[0]] = D[0]
    lex_scores = (X_lex @ tfidf.transform([q_lex_text]).T).toarray().ravel()
    sem_n = minmax_scale(sem_scores) if sem_scores.max()>0 else sem_scores
    lex_n = minmax_scale(lex_scores) if lex_scores.max()>0 else lex_scores
    s1 = w_sem*sem_n + w_lex*lex_n; shortlist_idx = np.argsort(s1)[::-1][:max(4*topk,50)]
    want_exp = {expand_token(s).lower() for s in skills+[devtype]}
    cov, ttl_bonus = [], []
    for txt, ttl in zip(courses.iloc[shortlist_idx]["text_lex"], courses.iloc[shortlist_idx]["title"].fillna("").str.lower()):
        hits = sum(1 for s in want_exp if s in txt)
        cov.append(hits/len(want_exp) if want_exp else 0)
        ttl_bonus.append(title_bonus if any(s in ttl for s in want_exp) else 0)
    cov = np.asarray(cov, dtype="float32")
    cov_n = minmax_scale(cov) if cov.max()>0 else cov
    blend = (w_sem*sem_n[shortlist_idx] + w_lex*lex_n[shortlist_idx] + w_cov*cov_n + np.asarray(ttl_bonus))
    order = np.argsort(blend)[::-1]; top_idx = [shortlist_idx[i] for i in order[:topk]]
    out = courses.iloc[top_idx][["title","url","source","level","duration","description_full","duration_hours"]].copy()
    out.insert(0,"DevType",devtype)
    out["score_sem"]=sem_n[top_idx]; out["score_lex"]=lex_n[top_idx]; out["coverage"]=cov_n[order][:len(out)].round(3); out["score"]=blend[order][:len(out)].round(4)
    return out.sort_values(by=["score","score_sem","score_lex","coverage","duration_hours","title"],ascending=[False,False,False,False,True,True]).reset_index(drop=True)

# -----------------------------
# 5) Run for all DevTypes
# -----------------------------
all_recs = [score_and_rank_for_row(r, topk=5) for _,r in df_skills.iterrows()]
recs_df = pd.concat(all_recs, ignore_index=True)

# Example
print(recs_df.head(15))
# recs_df.to_csv("recommendations_top5.csv", index=False)


                 DevType                                              title  \
0      Developer, mobile                                      Learn Next.js   
1      Developer, mobile           Frontend for Java Full Stack Development   
2      Developer, mobile           Frontend for Java Full Stack Development   
3      Developer, mobile           Frontend for Java Full Stack Development   
4      Developer, mobile                  Mobile Development and JavaScript   
5    Developer, back-end                Introduction to Databases in Python   
6    Developer, back-end              Create a Back-End App with JavaScript   
7    Developer, back-end  Scripting with Python and SQL for Data Enginee...   
8    Developer, back-end  Scripting with Python and SQL for Data Enginee...   
9    Developer, back-end                                  Learn Node-SQLite   
10  Developer, front-end                   Secure Full Stack MEAN Developer   
11  Developer, front-end                   Secure Fu

In [34]:
recs_df.head(15)

,DevType,title,url,source,level,duration,description_full,duration_hours,score_sem,score_lex,coverage,score
0,"Developer, mobile",Learn Next.js,https://www.codecademy.com/learn/learn-next-js,Codecademy,Advanced,12 hours,Unlock the full potential of Next.js in this c...,12.0,0.809574,0.901139,0.200,0.9262
1,"Developer, mobile",Frontend for Java Full Stack Development,https://www.coursera.org/learn/frontend-develo...,Coursera,Intermediate,3 week,This comprehensive course is designed to equip...,9.0,0.841396,0.628705,0.600,0.8963
2,"Developer, mobile",Frontend for Java Full Stack Development,https://www.coursera.org/learn/frontend-develo...,Coursera,Intermediate,3 week,This comprehensive course is designed to equip...,9.0,0.841396,0.628705,0.600,0.8963
3,"Developer, mobile",Frontend for Java Full Stack Development,https://www.coursera.org/learn/frontend-develo...,Coursera,Intermediate,3 week,This comprehensive course is designed to equip...,9.0,0.841396,0.628705,0.600,0.8963
4,"Developer, mobile",Mobile Development and JavaScript,https://www.coursera.org/learn/mobile-developm...,Coursera,Beginner,5 week,This course is the first in a series that will...,15.0,0.909462,0.581865,0.400,0.8884
5,"Developer, back-end",Introduction to Databases in Python,https://app.datacamp.com/learn/courses/introdu...,DataCamp,Intermediate,4 hr,"In this course, you'll learn the basics of usi...",4.0,0.896713,0.570483,1.000,0.9662
6,"Developer, back-end",Create a Back-End App with JavaScript,https://www.codecademy.com/learn/paths/create-...,Codecademy,Beginner,30 hours,Learn how to build back-end web APIs using Exp...,30.0,0.714555,0.883307,0.750,0.9446
7,"Developer, back-end",Scripting with Python and SQL for Data Enginee...,https://www.coursera.org/learn/scripting-with-...,Coursera,Intermediate,2 week,"In this third course of the Python, Bash and S...",6.0,0.835219,0.633921,0.750,0.9172
8,"Developer, back-end",Scripting with Python and SQL for Data Enginee...,https://www.coursera.org/learn/scripting-with-...,Coursera,Intermediate,2 week,"In this third course of the Python, Bash and S...",6.0,0.835219,0.633921,0.750,0.9172
9,"Developer, back-end",Learn Node-SQLite,https://www.codecademy.com/learn/learn-node-sq...,Codecademy,Intermediate,2 hours,This course teaches you how to integrate SQL i...,2.0,0.957878,0.537184,0.500,0.9146
